In [8]:
#from kafka import KafkaProducer
from wetterdienst import DWDObservationData, DWDObservationParameterSet, DWDObservationResolution
from pymongo import MongoClient

ImportError: cannot import name 'DWDObservationData' from 'wetterdienst' (/opt/conda/lib/python3.7/site-packages/wetterdienst/__init__.py)